# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Numerical-linear-algebra:-introduction" data-toc-modified-id="Numerical-linear-algebra:-introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Numerical linear algebra: introduction</a></div><div class="lev2 toc-item"><a href="#BLAS" data-toc-modified-id="BLAS-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>BLAS</a></div><div class="lev2 toc-item"><a href="#Memory-hierarchy-and-level-3-fraction" data-toc-modified-id="Memory-hierarchy-and-level-3-fraction-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Memory hierarchy and level-3 fraction</a></div><div class="lev2 toc-item"><a href="#Effect-of-data-layout" data-toc-modified-id="Effect-of-data-layout-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Effect of data layout</a></div><div class="lev2 toc-item"><a href="#BLAS-in-R" data-toc-modified-id="BLAS-in-R-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>BLAS in R</a></div><div class="lev2 toc-item"><a href="#Avoid-memory-allocation:-some-examples" data-toc-modified-id="Avoid-memory-allocation:-some-examples-15"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Avoid memory allocation: some examples</a></div>

In [1]:
versioninfo()

Julia Version 1.1.0
Commit 80516ca202 (2019-01-21 21:24 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin14.5.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code


# Numerical linear algebra: introduction

* Topics in numerical algebra: 
    - BLAS  
    - solve linear equations $\mathbf{A} \mathbf{x} = \mathbf{b}$
    - regression computations $\mathbf{X}^T \mathbf{X} \beta = \mathbf{X}^T \mathbf{y}$  
    - eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{x}$  
    - generalized eigen-problems $\mathbf{A} \mathbf{x} = \lambda \mathbf{B} \mathbf{x}$  
    - singular value decompositions $\mathbf{A} = \mathbf{U} \Sigma \mathbf{V}^T$  
    - iterative methods for numerical linear algebra    

* Except for the iterative methods, most of these numerical linear algebra tasks are implemented in the BLAS and LAPACK libraries. They form the **building blocks** of most statistical computing tasks (optimization, MCMC).

* Our major **goal** (or learning objectives) is to  
    1. know the complexity (flop count) of each task
    2. be familiar with the BLAS and LAPACK functions (what they do)  
    3. do **not** re-invent wheels by implementing these dense linear algebra subroutines by yourself  
    4. understand the need for iterative methods  
    5. apply appropriate numerical algebra tools to various statistical problems 

* All high-level languages (R, Matlab, Julia) call BLAS and LAPACK for numerical linear algebra. 
    - Julia offers more flexibility by exposing interfaces to many BLAS/LAPACK subroutines directly. See [documentation](https://docs.julialang.org/en/v1.1/stdlib/LinearAlgebra/#BLAS-Functions-1).

## BLAS

* BLAS stands for _basic linear algebra subprograms_. 

* See [netlib](http://www.netlib.org/blas/) for a complete list of standardized BLAS functions.

* There are many implementations of BLAS. 
    - [Netlib](http://www.netlib.org/blas/) provides a reference implementation.  
    - Matlab uses Intel's [MKL](https://software.intel.com/en-us/node/520724) (mathematical kernel libaries). **MKL implementation is the gold standard on market.** It is not open source but the compiled library is free for Linux and MacOS.    
    - Julia uses [OpenBLAS](https://github.com/xianyi/OpenBLAS). **OpenBLAS is the best open source implementation**.    

* There are 3 levels of BLAS functions.
    - [Level 1](http://www.netlib.org/blas/#_level_1): vector-vector operation
    - [Level 2](http://www.netlib.org/blas/#_level_2): matrix-vector operation
    - [Level 3](http://www.netlib.org/blas/#_level_3): matrix-matrix operation

| Level | Example Operation                      | Name        | Dimension                                 | Flops |  
|-------|----------------------------------------|-------------|-------------------------------------------|-------|
| 1     | $\alpha \gets \mathbf{x}^T \mathbf{y}$ | dot product | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ | $2n$  |  
| 1 | $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$ |  axpy           |  $\alpha \in \mathbb{R}$, $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$ |  $2n$    |  
| 2     | $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ |  gaxpy           |  $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                     |  $2mn$     |
| 2 | $\mathbf{A} \gets \mathbf{A} + \mathbf{y} \mathbf{x}^T$ | rank one update            |    $\mathbf{A} \in \mathbb{R}^{m \times n}$, $\mathbf{x} \in \mathbb{R}^n$, $\mathbf{y} \in \mathbb{R}^m$                                       | $2mn$      |
| 3     | $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$                                       |  matrix multiplication           |  $\mathbf{A} \in \mathbb{R}^{m \times p}$, $\mathbf{B} \in \mathbb{R}^{p \times n}$, $\mathbf{C} \in \mathbb{R}^{m \times n}$                                         | $2mnp$      |

* Typical BLAS functions support single precision (S), double precision (D), complex (C), and double complex (Z). 

> The form of a mathematical expression and the way the expression should be evaluated in actual practice may be quite different.

* Some operations _appear_ as level-3 but indeed are level-2.  
    - A common operation in statistics is column scaling or row scaling
    $$
    \begin{eqnarray*}
        \mathbf{A} &=& \mathbf{A} \mathbf{D} \quad \text{(column scaling)} \\
        \mathbf{A} &=& \mathbf{D} \mathbf{A} \quad \text{(row scaling)},
    \end{eqnarray*}
    $$
    where $\mathbf{D}$ is diagonal.  
    - These are essentially level-2 operations!

In [2]:
using BenchmarkTools, LinearAlgebra, Random

Random.seed!(123) # seed
n = 2000
A = rand(n, n) # n-by-n matrix
d = rand(n);  # n vector

In [3]:
Dfull = diagm(0 => d) # diagonal matrix with d as diagonal

2000×2000 Array{Float64,2}:
 0.140972  0.0       0.0       0.0        …  0.0       0.0       0.0     
 0.0       0.143596  0.0       0.0           0.0       0.0       0.0     
 0.0       0.0       0.612494  0.0           0.0       0.0       0.0     
 0.0       0.0       0.0       0.0480573     0.0       0.0       0.0     
 0.0       0.0       0.0       0.0           0.0       0.0       0.0     
 0.0       0.0       0.0       0.0        …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0           0.0       0.0       0.0     
 0.0       0.0       0.0       0.0           0.0       0.0       0.0     
 0.0       0.0       0.0       0.0           0.0       0.0       0.0     
 0.0       0.0       0.0       0.0           0.0       0.0       0.0     
 0.0       0.0       0.0       0.0        …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0           0.0       0.0       0.0     
 0.0       0.0       0.0       0.0           0.0       0.0       0.0     
 ⋮        

In [4]:
# this is calling BLAS routine for matrix multiplication: O(n^3) flops
@benchmark $A * $Dfull

BenchmarkTools.Trial: 
  memory estimate:  30.52 MiB
  allocs estimate:  2
  --------------
  minimum time:     105.269 ms (0.26% GC)
  median time:      109.647 ms (2.24% GC)
  mean time:        111.194 ms (2.95% GC)
  maximum time:     139.484 ms (26.13% GC)
  --------------
  samples:          46
  evals/sample:     1

In [5]:
Ddiag = Diagonal(d)

2000×2000 Diagonal{Float64,Array{Float64,1}}:
 0.140972   ⋅         ⋅         ⋅         …   ⋅         ⋅         ⋅      
  ⋅        0.143596   ⋅         ⋅             ⋅         ⋅         ⋅      
  ⋅         ⋅        0.612494   ⋅             ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅        0.0480573      ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅             ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅         …   ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅             ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅             ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅             ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅             ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅         …   ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅             ⋅         ⋅         ⋅      
  ⋅         ⋅         ⋅         ⋅             ⋅         ⋅         

In [6]:
# columnwise scaling: O(n^2) flops
@benchmark $A * $Ddiag

BenchmarkTools.Trial: 
  memory estimate:  30.52 MiB
  allocs estimate:  2
  --------------
  minimum time:     10.393 ms (3.15% GC)
  median time:      11.146 ms (21.84% GC)
  mean time:        11.437 ms (22.28% GC)
  maximum time:     51.828 ms (83.51% GC)
  --------------
  samples:          437
  evals/sample:     1

In [7]:
@which A * Ddiag

*(A::AbstractArray{T,2} where T, D::Diagonal) in LinearAlgebra at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/diagonal.jl:168

In [8]:
# in-place: avoid allocate space for result
# rmul!: compute matrix-matrix product AB, overwriting A, and return the result.
@benchmark rmul!(A, Ddiag)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.273 ms (0.00% GC)
  median time:      10.641 ms (0.00% GC)
  mean time:        11.273 ms (0.00% GC)
  maximum time:     19.741 ms (0.00% GC)
  --------------
  samples:          444
  evals/sample:     1

## Memory hierarchy and level-3 fraction

> **Key to high performance is effective use of memory hierarchy. True on all architectures.**

* Flop count is not the sole determinant of algorithm efficiency. Another important factor is data movement through the memory hierarchy.

<img src="./macpro_inside.png" width="400" align="center">

<img src="./cpu_die.png" width="400" align="center">  

<img src="http://images.bit-tech.net/content_images/2007/11/the_secrets_of_pc_memory_part_1/hei.png" width="400" align="center">

* Numbers everyone should know

| Operation                           | Time           |
|-------------------------------------|----------------|
| L1 cache reference                  | 0.5 ns         |
| L2 cache reference                  | 7 ns           |
| Main memory reference               | 100 ns         |
| Read 1 MB sequentially from memory  | 250,000 ns     |
| Read 1 MB sequentially from SSD     | 1,000,000 ns   |  
| Read 1 MB sequentially from disk    | 20,000,000 ns  |


<!-- | Operation                           | Time           | -->
<!-- |-------------------------------------|----------------| -->
<!-- | L1 cache reference                  | 0.5 ns         | -->
<!-- | Branch mispredict                   | 5 ns           | -->
<!-- | L2 cache reference                  | 7 ns           | -->
<!-- | Mutex lock/unlock                   | 100 ns         | -->
<!-- | Main memory reference               | 100 ns         | -->
<!-- | Compress 1K bytes with Zippy        | 10,000 ns      | -->
<!-- | Send 2K bytes over 1 Gbps network   | 20,000 ns      | -->
<!-- | Read 1 MB sequentially from memory  | 250,000 ns     | -->
<!-- | Round trip within same datacenter   | 500,000 ns     | -->
<!-- | Disk seek                           | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from network | 10,000,000 ns  | -->
<!-- | Read 1 MB sequentially from disk    | 30,000,000 ns  | -->
<!-- | Send packet CA->Netherlands->CA     | 150,000,000 ns | -->

   Source: <https://gist.github.com/jboner/2841832>  

* For example, Xeon X5650 CPU has a theoretical throughput of 128 DP GFLOPS but a max memory bandwidth of 32GB/s.  

* Can we keep CPU cores busy with enough deliveries of matrix data and ship the results to memory fast enough to avoid backlog?  
Answer: use **high-level BLAS** as much as possible.

| BLAS | Dimension | Mem. Refs. | Flops  | Ratio |
|--------------------------------|------------------------------------------------------------|------------|--------|-------|
| Level 1: $\mathbf{y} \gets \mathbf{y} + \alpha \mathbf{x}$     | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$                                           | $3n$       | $2n$   | 3:2   |
| Level 2: $\mathbf{y} \gets \mathbf{y} + \mathbf{A} \mathbf{x}$ | $\mathbf{x}, \mathbf{y} \in \mathbb{R}^n$, $\mathbf{A} \in \mathbb{R}^{n \times n}$ | $n^2$      | $2n^2$ | 1:2   |
| Level 3: $\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}$ | $\mathbf{A}, \mathbf{B}, \mathbf{C} \in\mathbb{R}^{n \times n}$                    | $4n^2$     | $2n^3$ | 2:n |  

* Higher level BLAS (3 or 2) make more effective use of arithmetic logic units (ALU) by keeping them busy. **Surface-to-volume** effect.  
See [Dongarra slides](https://www.samsi.info/wp-content/uploads/2017/02/SAMSI-0217_Dongarra.pdf).

<img src="./blas_throughput.png" width="500" align="center"/>

* A distinction between LAPACK and LINPACK (older version of R uses LINPACK) is that LAPACK makes use of higher level BLAS as much as possible (usually by smart partitioning) to increase the so-called **level-3 fraction**.

## Effect of data layout

* Data layout in memory affects algorithmic efficiency too. It is much faster to move chunks of data in memory than retrieving/writing scattered data.

* Storage mode: **column-major** (Fortran, Matlab, R, Julia) vs **row-major** (C/C++).

* **Cache line** is the minimum amount of cache which can be loaded and stored to memory.
    - x86 CPUs: 64 bytes  
    - ARM CPUs: 32 bytes

<img src="https://patterns.eecs.berkeley.edu/wordpress/wp-content/uploads/2013/04/dense02.png" width="500" align="center"/>

* Accessing column-major stored matrix by rows ($ij$ looping) causes lots of **cache misses**.

* Take matrix multiplication as an example 
$$ 
\mathbf{C} \gets \mathbf{C} + \mathbf{A} \mathbf{B}, \quad \mathbf{A} \in \mathbb{R}^{m \times p}, \mathbf{B} \in \mathbb{R}^{p \times n}, \mathbf{C} \in \mathbb{R}^{m \times n}.
$$
Assume the storage is column-major, such as in Julia. There are 6 variants of the algorithms according to the order in the triple loops. 
    - `jki` or `kji` looping:
        ```julia
        # inner most loop
        for i = 1:m
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```  
    - `ikj` or `kij` looping:
        ```julia
        # inner most loop        
        for j = 1:n
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```  
    - `ijk` or `jik` looping:
        ```julia
        # inner most loop        
        for k = 1:p
            C[i, j] = C[i, j] + A[i, k] * B[k, j]
        end
        ```
* We pay attention to the innermost loop, where the vector calculation occurs. The associated **stride** when accessing the three matrices in memory (assuming column-major storage) is  

| Variant        | A Stride | B Stride | C Stride |
|----------------|----------|----------|----------|
| $jki$ or $kji$ | Unit     | 0        | Unit     |
| $ikj$ or $kij$ | 0        | Non-Unit | Non-Unit |
| $ijk$ or $jik$ | Non-Unit | Unit     | 0        |       
Apparently the variants $jki$ or $kji$ are preferred.

In [9]:
"""
    matmul_by_loop!(A, B, C, order)

Overwrite `C` by `A * B`. `order` indicates the looping order for triple loop.
"""
function matmul_by_loop!(A::Matrix, B::Matrix, C::Matrix, order::String)
    
    m = size(A, 1)
    p = size(A, 2)
    n = size(B, 2)
    fill!(C, 0)
    
    if order == "jki"
        for j = 1:n, k = 1:p, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kji"
        for k = 1:p, j = 1:n, i = 1:m
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ikj"
        for i = 1:m, k = 1:p, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end

    if order == "kij"
        for k = 1:p, i = 1:m, j = 1:n
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "ijk"
        for i = 1:m, j = 1:n, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
    if order == "jik"
        for j = 1:n, i = 1:m, k = 1:p
            C[i, j] += A[i, k] * B[k, j]
        end
    end
    
end

using Random

Random.seed!(123)
m, n, p = 2000, 100, 2000
A = rand(m, n)
B = rand(n, p)
C = zeros(m, p);

* $jki$ and $kji$ looping:

In [10]:
using BenchmarkTools

@benchmark matmul_by_loop!($A, $B, $C, "jki")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     279.904 ms (0.00% GC)
  median time:      287.860 ms (0.00% GC)
  mean time:        287.899 ms (0.00% GC)
  maximum time:     298.282 ms (0.00% GC)
  --------------
  samples:          18
  evals/sample:     1

In [11]:
@benchmark matmul_by_loop!($A, $B, $C, "kji")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     336.070 ms (0.00% GC)
  median time:      347.895 ms (0.00% GC)
  mean time:        351.619 ms (0.00% GC)
  maximum time:     381.221 ms (0.00% GC)
  --------------
  samples:          15
  evals/sample:     1

* $ikj$ and $kij$ looping:

In [12]:
@benchmark matmul_by_loop!($A, $B, $C, "ikj")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.635 s (0.00% GC)
  median time:      2.641 s (0.00% GC)
  mean time:        2.641 s (0.00% GC)
  maximum time:     2.647 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1

In [13]:
@benchmark matmul_by_loop!($A, $B, $C, "kij")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.707 s (0.00% GC)
  median time:      2.716 s (0.00% GC)
  mean time:        2.716 s (0.00% GC)
  maximum time:     2.725 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1

* $ijk$ and $jik$ looping:

In [14]:
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.032 s (0.00% GC)
  median time:      1.052 s (0.00% GC)
  mean time:        1.050 s (0.00% GC)
  maximum time:     1.063 s (0.00% GC)
  --------------
  samples:          5
  evals/sample:     1

In [15]:
@benchmark matmul_by_loop!($A, $B, $C, "ijk")

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.036 s (0.00% GC)
  median time:      1.047 s (0.00% GC)
  mean time:        1.045 s (0.00% GC)
  maximum time:     1.053 s (0.00% GC)
  --------------
  samples:          5
  evals/sample:     1

* Julia wraps BLAS library for matrix multiplication. We see BLAS library wins hands down (multi-threading, Strassen algorithm, higher level-3 fraction by block outer product).

In [16]:
@benchmark mul!($C, $A, $B)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.102 ms (0.00% GC)
  median time:      7.498 ms (0.00% GC)
  mean time:        8.026 ms (0.00% GC)
  maximum time:     12.123 ms (0.00% GC)
  --------------
  samples:          622
  evals/sample:     1

In [17]:
# direct call of BLAS wrapper function
@benchmark LinearAlgebra.BLAS.gemm!('N', 'N', 1.0, $A, $B, 0.0, $C)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.129 ms (0.00% GC)
  median time:      8.138 ms (0.00% GC)
  mean time:        8.711 ms (0.00% GC)
  maximum time:     18.798 ms (0.00% GC)
  --------------
  samples:          573
  evals/sample:     1

To appreciate the efforts in an optimized BLAS implementation such as OpenBLAS (evolved from GotoBLAS), see the [Quora question](https://www.quora.com/What-algorithm-does-BLAS-use-for-matrix-multiplication-Of-all-the-considerations-e-g-cache-popular-instruction-sets-Big-O-etc-which-one-turned-out-to-be-the-primary-bottleneck), especially the [video](https://youtu.be/JzNpKDW07rw). Bottomline is 

> **Get familiar with (good implementations of) BLAS/LAPACK and use them as much as possible.**

## BLAS in R

* **Tip for R user**. Standard R distribution from CRAN uses a very out-dated BLAS/LAPACK library.

In [18]:
using RCall

R"""
library("microbenchmark")
microbenchmark($A %*% $B)
"""

RObject{VecSxp}
Unit: milliseconds
                expr      min       lq     mean   median       uq     max neval
 `#JL`$A %*% `#JL`$B 229.4794 239.3843 244.6556 243.7473 249.5183 298.638   100


* Re-build R from source using OpenBLAS or MKL will immediately boost linear algebra performance in R. Google `build R using MKL` to get started. Similarly we can build Julia using MKL.

* Matlab uses MKL. Usually it's very hard to beat Matlab in terms of linear algebra.

In [19]:
using MATLAB

mat"""
f = @() $A * $B;
timeit(f)
"""

0.012180794699500001

## Avoid memory allocation: some examples

* Transposing matrix is an expensive memory operation.  
    - In R, the command 
        ```R
        t(A) %*% x
        ```
    will first transpose `A` then perform matrix multiplication, causing unnecessary memory allocation
    - Julia is smart to avoid transposing matrix if possible.

In [20]:
Random.seed!(123)

n = 1000
A = rand(n, n)
x = rand(n);

In [21]:
# dispatch BLAS
# does *not* actually transpose the matrix
@benchmark transpose($A) * $x

BenchmarkTools.Trial: 
  memory estimate:  7.94 KiB
  allocs estimate:  1
  --------------
  minimum time:     89.576 μs (0.00% GC)
  median time:      130.354 μs (0.00% GC)
  mean time:        150.637 μs (0.00% GC)
  maximum time:     988.192 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [22]:
# pre-allocate result
out = zeros(size(A, 2))
@benchmark mul!($out, $A, $x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     97.222 μs (0.00% GC)
  median time:      132.195 μs (0.00% GC)
  mean time:        151.802 μs (0.00% GC)
  maximum time:     876.984 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [23]:
# or call BLAS wrapper directly
@benchmark LinearAlgebra.BLAS.gemv!('N', 1.0, $A, $x, 0.0, $out)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     96.573 μs (0.00% GC)
  median time:      121.846 μs (0.00% GC)
  mean time:        135.712 μs (0.00% GC)
  maximum time:     425.209 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

* [Broadcasting](https://docs.julialang.org/en/v1/base/arrays/#Broadcast-and-vectorization-1) in Julia achieves vectorized code without creating intermediate arrays.

    Suppose we want to calculate elementsize maximum of absolute values of two large arrays. In R or Matlab, the command
```r
max(abs(X), abs(Y))
```
will create two intermediate arrays and then one result array.

In [24]:
using RCall
Random.seed!(123)
X, Y = rand(1000, 1000), rand(1000, 1000)

R"""
library(microbenchmark)
microbenchmark(max(abs($X), abs($Y)))
"""

RObject{VecSxp}
Unit: milliseconds
                            expr      min       lq     mean   median       uq
 max(abs(`#JL`$X), abs(`#JL`$Y)) 5.733193 6.385078 8.143044 6.976127 8.490197
      max neval
 25.65402   100


In Julia, dot operations are fused so no intermediate arrays are created.

In [25]:
# no intermediate arrays created, only result array created
@benchmark max.(abs.($X), abs.($Y))

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  2
  --------------
  minimum time:     1.678 ms (0.00% GC)
  median time:      1.919 ms (0.00% GC)
  mean time:        2.547 ms (27.04% GC)
  maximum time:     5.085 ms (50.23% GC)
  --------------
  samples:          1958
  evals/sample:     1

Pre-allocating result array gets rid of memory allocation at all.

In [26]:
# no memory allocation at all!
Z = zeros(size(X)) # zero matrix of same size as X
@benchmark $Z .= max.(abs.($X), abs.($Y)) # .= (vs =) is important!

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.141 ms (0.00% GC)
  median time:      1.331 ms (0.00% GC)
  mean time:        1.380 ms (0.00% GC)
  maximum time:     3.271 ms (0.00% GC)
  --------------
  samples:          3602
  evals/sample:     1

* [View](https://docs.julialang.org/en/v1/base/arrays/#Views-(SubArrays-and-other-view-types)-1) avoids creating extra copy of matrix data.

In [27]:
Random.seed!(123)
A = randn(1000, 1000)

# sum entries in a sub-matrix
@benchmark sum($A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 
  memory estimate:  488.39 KiB
  allocs estimate:  2
  --------------
  minimum time:     64.866 μs (0.00% GC)
  median time:      275.892 μs (0.00% GC)
  mean time:        254.370 μs (14.08% GC)
  maximum time:     2.602 ms (87.65% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [28]:
# view avoids creating a separate sub-matrix
@benchmark sum(@view $A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 
  memory estimate:  208 bytes
  allocs estimate:  7
  --------------
  minimum time:     118.343 μs (0.00% GC)
  median time:      125.083 μs (0.00% GC)
  mean time:        126.912 μs (0.00% GC)
  maximum time:     385.135 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

The [`@views`](https://docs.julialang.org/en/v1/base/arrays/#Base.@views) macro, which can be useful in [some operations](https://discourse.julialang.org/t/why-is-a-manual-in-place-addition-so-much-faster-than-and-on-range-indexed-arrays/3302).

In [29]:
@benchmark @views sum($A[1:2:500, 1:2:500])

BenchmarkTools.Trial: 
  memory estimate:  208 bytes
  allocs estimate:  7
  --------------
  minimum time:     124.948 μs (0.00% GC)
  median time:      128.717 μs (0.00% GC)
  mean time:        131.780 μs (0.00% GC)
  maximum time:     376.983 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1